In [1]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory

import boto3
from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
# Get Retriever
def load_vector_db():
    # load db
    embeddings_model = OpenAIEmbeddings()
    vectorestore = FAISS.load_local('./db/faiss', embeddings_model, allow_dangerous_deserialization=True )
    retriever = vectorestore.as_retriever()
    return retriever

retriever = load_vector_db()

In [3]:
# Get LLM
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

model_kwargs =  { 
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
    #"stop_sequences": ["\n\nHuman"],
}

llm = ChatBedrock(
    client=bedrock_runtime,
    model_id=model_id,
    model_kwargs=model_kwargs,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [4]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [5]:
# prompt = hub.pull("rlm/rag-prompt")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an assistant for question-answering tasks. 
            Use the following pieces of retrieved context to answer the question. 
            If you don't know the answer, just say that you don't know. 
            Use three sentences maximum and keep the answer concise.
            \n\n
            {context}"""
        ),
        (
            "human", "{question}",

        )
    ]
)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [6]:
rag_chain.invoke("회원 가입 규정은")

카카오 계정 약관에 따르면, 회원 가입 규정은 다음과 같습니다.

1. 회원 가입 시 허위 사실을 기재하거나 다른 사람의 계정을 도용하는 행위는 금지됩니다.

2. 회사는 서비스 제공을 위해 필요한 경우 회원 가입을 제한할 수 있습니다. 예를 들어 서비스 설비 용량에 여유가 없거나, 기술적 문제가 있는 경우 등입니다. 

3. 회원 자격 정지 기간 중에는 임의로 이용 계약을 해지하고 재가입할 수 없습니다.

회원 가입 시에는 정확한 정보를 제공하고, 회사의 정책과 지침을 준수해야 합니다. 회사는 서비스 운영을 위해 필요한 경우 회원 가입을 제한할 수 있는 권한이 있습니다.

'카카오 계정 약관에 따르면, 회원 가입 규정은 다음과 같습니다.\n\n1. 회원 가입 시 허위 사실을 기재하거나 다른 사람의 계정을 도용하는 행위는 금지됩니다.\n\n2. 회사는 서비스 제공을 위해 필요한 경우 회원 가입을 제한할 수 있습니다. 예를 들어 서비스 설비 용량에 여유가 없거나, 기술적 문제가 있는 경우 등입니다. \n\n3. 회원 자격 정지 기간 중에는 임의로 이용 계약을 해지하고 재가입할 수 없습니다.\n\n회원 가입 시에는 정확한 정보를 제공하고, 회사의 정책과 지침을 준수해야 합니다. 회사는 서비스 운영을 위해 필요한 경우 회원 가입을 제한할 수 있는 권한이 있습니다.'

### Chat hisotry 사용하기

In [7]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Chain with chat history

In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [9]:
chat_history = []

question = "회원 가입 규정은"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(ai_msg_1["answer"])])

second_question = "카카오콘이 뭐냐"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

요약하면, 카카오 계정 약관에 따르면 회원 가입 시 다음 사항을 준수해야 합니다.

1. 이용 신청 시 허위 사실을 기재하거나 다른 사람의 계정을 도용해서는 안 됩니다. 

2. 회사는 서비스 제공을 위해 필요한 경우 가입을 제한할 수 있습니다. 예를 들어 서비스 설비 용량 부족, 기술적 문제, 재정적 필요성 등의 이유로 가입을 제한할 수 있습니다.

3. 회원 자격 정지 기간 중에는 새로 가입할 수 없습니다.

회원 가입 시에는 진실된 정보를 제공하고, 회사의 합리적인 가입 제한 사유에 따라야 합니다.카카오콘(Kakao Con)은 카카오가 매년 개최하는 개발자 컨퍼런스입니다. 카카오의 새로운 기술과 서비스를 소개하고, 개발자들이 정보를 공유하고 네트워킹할 수 있는 행사입니다. 카카오콘에서는 카카오의 다양한 서비스와 기술 동향, 개발 사례 등을 살펴볼 수 있습니다. 개발자들에게 유용한 정보와 인사이트를 제공하는 것이 주요 목적입니다.카카오콘은 카카오가 회원들에게 제공하는 가상의 혜택 포인트입니다. 약관에 따르면 카카오콘에 대해 다음과 같이 설명하고 있습니다.

1. 카카오콘은 회사가 회원의 개별 서비스 이용 과정에서 지급하는 혜택입니다. 회원은 회사가 정한 방법에 따라 카카오콘을 사용할 수 있습니다.

2. 카카오콘은 재산적 가치가 없으며, 현금으로 환전하거나 환불받을 수 없습니다. 

3. 카카오콘으로 이용 가능한 서비스와 혜택은 회사 정책과 제휴 관계에 따라 수시로 변경될 수 있습니다.

4. 회사는 운영 목적상 필요한 경우 카카오콘을 조정, 이용 제한, 소멸 등의 조치를 취할 수 있습니다.

요약하면 카카오콘은 현금가치가 없는 가상의 포인트로, 회사가 정한 바에 따라 일정 혜택을 제공받을 수 있습니다.카카오콘은 카카오가 회원들에게 제공하는 가상의 혜택 포인트입니다. 약관에 따르면 카카오콘에 대해 다음과 같이 설명하고 있습니다.

1. 카카오콘은 회사가 회원의 개별 서비스 이용 과정에서 지급하는 혜택입니다. 회원은 회사가 정한 방법에 따라 카카오콘을 사용할